In [ ]:
!pip install torch torchvision albumentations tqdm codecarbon optuna

In [ ]:
import albumentations.pytorch
import datasets as ds
import utils
import torch as t
import torchvision as tv
import torchvision.tv_tensors as tvt
import albumentations as A
from tqdm.auto import tqdm
import torchmetrics as tm
import optuna
from codecarbon import EmissionsTracker


def objective(trial: optuna.Trial) -> float:
    x2, y2 = ds.extract_all('./datasets/30/annotations.xml')
    x3, y3 = ds.extract_all('./datasets/60/annotations.xml')
    x4, y4 = ds.extract_all('./datasets/90/annotations.xml')
    x6, y6 = ds.extract_all('./datasets/clear2/annotations.xml')
    x7, y7 = ds.extract_all('./datasets/clear3/annotations.xml')


    x0, y0 = ds.extract_all('./datasets/090/annotations.xml')
    x1, y1 = ds.extract_all('./datasets/190/annotations.xml')
    x5, y5 = ds.extract_all('./datasets/clear1/annotations.xml')

    augmentations = []

    if trial.suggest_categorical("enable_horizontal_flip", [True, False]):
        augmentations.append(A.HorizontalFlip(p=0.5))

    if trial.suggest_categorical("enable_color_jitter", [True, False]):
        augmentations.append(A.ColorJitter(
            brightness=trial.suggest_float("color_jitter_brightness", 0.1, 0.5),
            contrast=trial.suggest_float("color_jitter_contrast", 0.1, 0.5),
            saturation=trial.suggest_float("color_jitter_saturation", 0.1, 0.5),
            hue=trial.suggest_float("color_jitter_hue", 0.1, 0.2)
        ))

    if trial.suggest_categorical("enable_affine", [True, False]):
        kwords = {}
        if trial.suggest_categorical("enable_affine_scale", [True, False]):
            affine_scale_min = trial.suggest_float("affine_scale_min", 0.05, 0.2)
            affine_scale_max = trial.suggest_float("affine_scale_max", affine_scale_min, 0.5)
            kwords["scale"] = (affine_scale_min, affine_scale_max)

        if trial.suggest_categorical("enable_affine_rotate", [True, False]):
            affine_rotate_min = trial.suggest_float("affine_rotate_min", -20, -15)
            affine_rotate_max = trial.suggest_float("affine_rotate_max", 15, 20)
            kwords["rotate"] = (affine_rotate_min, affine_rotate_max)

        if trial.suggest_categorical("enable_affine_shear", [True, False]):
            affine_shear_min = trial.suggest_float("affine_shear_min", -20, -15)
            affine_shear_max = trial.suggest_float("affine_shear_max", 15, 20)
            kwords["shear"] = (affine_shear_min, affine_shear_max)

        augmentations.append(A.Affine(
            **kwords,
            p=0.5
        ))

    train_ds = ds.CustomImageDataset([*x2, *x3, *x4, *x6, *x7], [*y2, *y3, *y4, *y6, *y7],
                                     transform=A.Compose([
                                         *augmentations,
                                         albumentations.pytorch.ToTensorV2()
                                     ],
                                         bbox_params=ds.albumentations_params))

    val_ds = ds.CustomImageDataset([*x0, *x1, *x5], [*y0, *y1, *y5])

    dl_train = t.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=utils.unroller)
    dl_val = t.utils.data.DataLoader(val_ds, batch_size=4, shuffle=True, collate_fn=utils.unroller)

    device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')

    model = tv.models.detection.fasterrcnn_resnet50_fpn(weights='COCO_V1', backbone_weights='IMAGENET1K_V2').to(device)

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = tv.models.detection.faster_rcnn.FastRCNNPredictor(
        in_channels=model.roi_heads.box_predictor.cls_score.in_features,
        num_classes=2
    ).to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = t.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    metric = tm.detection.mean_ap.MeanAveragePrecision(
        box_format="xyxy",
        iou_thresholds=[0.5],
    )

    num_epochs = 3
    val_map = 0.0

    for epoch in range(num_epochs):
        # ---- TRAIN ----
        model.train()
        train_loss = 0

        dl_train_tqdm = tqdm(dl_train, desc=f"Train Epoch {epoch + 1}", leave=True)

        for images, targets in dl_train_tqdm:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            total_loss = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            train_loss += total_loss.item()

            dl_train_tqdm.set_postfix(loss=total_loss.item())

        train_loss /= len(dl_train)

        # ---- VALIDATION ----
        val_loss = 0
        metric.reset()

        with t.no_grad():
            for images, targets in dl_val:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                # loss
                model.train()
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                val_loss += losses

                # predictions
                model.eval()
                pred = model(images)
                metric.update(pred, targets)

        val_loss /= len(dl_val)  # avg loss

        val_metrics = metric.compute()
        val_map = val_metrics["map"].item()

        print(f"epoch={epoch + 1}; train_loss={train_loss:.4f}; val_loss={val_loss:.4f}; val_map={val_map:.4f}")

    return val_map


tracker = EmissionsTracker(log_level="info")
tracker.start()

study = optuna.create_study(direction="maximize", study_name="hyperparam_strats")
study.optimize(objective, n_trials=30)

emissions = tracker.stop()
print(f"Estimated CO2 emissions: {emissions:.6f} kg")


[codecarbon WARNING @ 10:10:23] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 10:10:23] [setup] RAM Tracking...
[codecarbon INFO @ 10:10:23] [setup] CPU Tracking...
[codecarbon WARNING @ 10:10:24] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 10:10:24] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-14900HX
[codecarbon WARNING @ 10:10:24] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:10:24] [setup] GPU Tracking...
[codecarbon INFO @ 10:10:24] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:10:24] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 10:10:24] >>> Tracker's metadata:
[co

Train Epoch 1:   0%|          | 0/84 [00:00<?, ?it/s]

[codecarbon INFO @ 10:10:26] Energy consumed for RAM : 0.006834 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:10:26] Delta energy consumed for CPU with constant : 0.003668 kWh, power : 880.0 W
[codecarbon INFO @ 10:10:26] Energy consumed for All CPU : 0.300758 kWh
[codecarbon INFO @ 10:10:26] Energy consumed for all GPUs : 0.011592 kWh. Total GPU Power : 14.640666260513598 W
[codecarbon INFO @ 10:10:26] 0.319185 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:10:28] Energy consumed for RAM : 0.001500 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:10:28] Delta energy consumed for CPU with constant : 0.003669 kWh, power : 880.0 W
[codecarbon INFO @ 10:10:28] Energy consumed for All CPU : 0.066023 kWh
[codecarbon INFO @ 10:10:28] Energy consumed for all GPUs : 0.004254 kWh. Total GPU Power : 23.970956962593576 W
[codecarbon INFO @ 10:10:28] 0.071777 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:10:30] Energy consumed for RAM : 0.007334 kWh. RAM Power

epoch=1; train_loss=0.1810; val_loss=0.4891; val_map=0.0266


Train Epoch 2:   0%|          | 0/84 [00:00<?, ?it/s]

...

Train Epoch 1:   0%|          | 0/84 [00:00<?, ?it/s]

[codecarbon INFO @ 10:48:02] Energy consumed for RAM : 0.016666 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:48:02] Delta energy consumed for CPU with constant : 0.003670 kWh, power : 880.0 W
[codecarbon INFO @ 10:48:02] Energy consumed for All CPU : 0.733435 kWh
[codecarbon INFO @ 10:48:02] Energy consumed for all GPUs : 0.053539 kWh. Total GPU Power : 75.79898843571235 W
[codecarbon INFO @ 10:48:02] 0.803640 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:48:02] 0.078553 g.CO2eq/s mean an estimation of 2,477.2462423453826 kg.CO2eq/year
[codecarbon INFO @ 10:48:03] Energy consumed for RAM : 0.015083 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:48:03] Delta energy consumed for CPU with constant : 0.003665 kWh, power : 880.0 W
[codecarbon INFO @ 10:48:03] Energy consumed for All CPU : 0.663746 kWh
[codecarbon INFO @ 10:48:03] Energy consumed for all GPUs : 0.051583 kWh. Total GPU Power : 74.7462650917072 W
[codecarbon INFO @ 10:48:03] 0.730413 kWh of electricity use

epoch=1; train_loss=0.1611; val_loss=1.2174; val_map=0.0322


Train Epoch 2:   0%|          | 0/84 [00:00<?, ?it/s]

[codecarbon INFO @ 10:49:26] Energy consumed for RAM : 0.012999 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:49:26] Delta energy consumed for CPU with constant : 0.003669 kWh, power : 880.0 W
[codecarbon INFO @ 10:49:26] Energy consumed for All CPU : 0.572048 kWh
[codecarbon INFO @ 10:49:26] Energy consumed for all GPUs : 0.047349 kWh. Total GPU Power : 79.11044922887686 W
[codecarbon INFO @ 10:49:26] 0.632396 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:49:27] Energy consumed for RAM : 0.019834 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:49:27] Delta energy consumed for CPU with constant : 0.003670 kWh, power : 880.0 W
[codecarbon INFO @ 10:49:27] Energy consumed for All CPU : 0.872852 kWh
[codecarbon INFO @ 10:49:27] Energy consumed for all GPUs : 0.058954 kWh. Total GPU Power : 78.89682270642558 W
[codecarbon INFO @ 10:49:27] 0.951640 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:49:30] Energy consumed for RAM : 0.014500 kWh. RAM Power :